In [109]:
import sys,os,json
import collections,math
import time,datetime,pytz
import numpy as np
import pandas as pd
import pickle
from scipy import interpolate, io
import matplotlib.pyplot as plt
from collections import Counter
from matplotlib.colors import ListedColormap
import seaborn as sns
import operator
from sklearn.utils import resample, shuffle
from sklearn.preprocessing import StandardScaler
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

## Audio Emotion Recognition Data

## Emotion-fbank data

In [91]:
path_fbank = 'data/audio-emotion/Emotions-fbanks/'

In [92]:
for filename in os.listdir(path_fbank):
    print(filename)

EMOTION_BOREDOM-1_CL001-CD2.plp.txt
EMOTION_NEUTRAL-TETE-1_GG002_CD1.plp.txt
EMOTION_NEUTRAL-NOR-1_JG001-CD3.plp.txt
EMOTION_NEUTRAL-CON-2_MK002-CD4.plp.txt
EMOTION_HOTANGER-1_GG001-CD2.plp.txt
EMOTION_DISGUST-1_CC001_CD1.plp.txt
EMOTION_SADNESS-1_CL001-CD2.plp.txt
EMOTION_HAPPY-1_MM001-CD5.plp.txt
EMOTION_PANIC-1_CC001_CD1.plp.txt
EMOTION_BOREDOM-1_CC001_CD1.plp.txt
EMOTION_NEUTRAL-TETE-1_MM002-CD5.plp.txt
EMOTION_BOREDOM-1_JG001-CD3.plp.txt
EMOTION_HOTANGER-1_MK001-CD2.plp.txt
EMOTION_PASSIVE-1_MM002-CD5.plp.txt
EMOTION_DOMINANT-1_MF002-CD3.plp.txt
EMOTION_NEUTRAL-TETE-2_CC002_CD1.plp.txt
EMOTION_INTEREST-1_JG001-CD3.plp.txt
EMOTION_NEUTRAL-CON-1_MK002-CD4.plp.txt
EMOTION_SADNESS-1_JG001-CD3.plp.txt
EMOTION_ELATION-1_MF001-CD4.plp.txt
EMOTION_HOTANGER-1_MF001-CD4.plp.txt
EMOTION_NEUTRAL-DIS-1_GG002_CD1.plp.txt
EMOTION_DISGUST-1_MM001-CD5.plp.txt
EMOTION_PANIC-1_GG001-CD2.plp.txt
EMOTION_NEUTRAL-CON-1_CC002_CD1.plp.txt
EMOTION_HOTANGER-1_JG001-CD3.plp.txt
EMOTION_PASSIVE-1_MF002-CD3.p

In [116]:
def appendJsonFromTxt(data, full_path):
    with open(full_path, 'r') as f_read:
        for line in f_read:
            # line_json is list object / and line is str
            line_json = json.loads(line)
            data.append(line_json) # add [500x24] feature vector
    
    return None

def splitByCV(X, y, rand):
    X, y = shuffle(X, y, random_state=rand)

    train_ratio = 0.8
    valid_ratio = 0.1
    cut_train = int(X.shape[0] * train_ratio)
    cut_valid = int(X.shape[0] * (train_ratio+valid_ratio))
    
    X_train = X[:cut_train]
    y_train = y[:cut_train]
    
    X_valid = X[cut_train:cut_valid]
    y_valid = y[cut_train:cut_valid]
    
    X_test = X[cut_valid:]
    y_test = y[cut_valid:]
    
    # change the shape from 3D to 2D [n_samplesx500x24] => [n_samplesx(500*24)]
    X_train = np.reshape(X_train, (X_train.shape[0],-1) )
    X_valid = np.reshape(X_valid, (X_valid.shape[0],-1) )
    X_test = np.reshape(X_test, (X_test.shape[0],-1) )
    
    X_train = StandardScaler().fit_transform(X_train)
    X_valid = StandardScaler().fit_transform(X_valid)
    X_test = StandardScaler().fit_transform(X_test)
    
    # change the shape from 2D to 3D [n_samplesx(500*24)] => [n_samplesx500x24]
    X_train = np.reshape(X_train, (X_train.shape[0],500,-1) )
    X_valid = np.reshape(X_valid, (X_valid.shape[0],500,-1) )
    X_test = np.reshape(X_test, (X_test.shape[0],500,-1) )
    
    # printout resulting y label count
    print('y_train: ', Counter(y_train))
    print('y_valid: ', Counter(y_valid))
    print('y_test: ', Counter(y_test))
    
    return X_train, y_train, X_valid, y_valid, X_test, y_test

In [95]:
data0 = []
data1 = []
data2 = []
data3 = []
data4 = []
data5 = []
data6 = []
data7 = []
data8 = []
data9 = []
data10 = []
data11 = []
data12 = []
data13 = []

for filename in os.listdir(path_fbank):
    if filename[:11] == 'EMOTION_NEU':
        if filename[:19] == 'EMOTION_NEUTRAL-NOR':
            appendJsonFromTxt(data0, path_fbank + filename)
        elif filename[:19] == 'EMOTION_NEUTRAL-CON':
            appendJsonFromTxt(data1, path_fbank + filename)
        elif filename[:19] == 'EMOTION_NEUTRAL-DIS':
            appendJsonFromTxt(data2, path_fbank + filename)
        elif filename[:19] == 'EMOTION_NEUTRAL-TET':
            appendJsonFromTxt(data3, path_fbank + filename)
    elif filename[:11] == 'EMOTION_BOR':
        appendJsonFromTxt(data4, path_fbank + filename)
    elif filename[:11] == 'EMOTION_PAS':
        appendJsonFromTxt(data5, path_fbank + filename)
    elif filename[:11] == 'EMOTION_DIS':
        appendJsonFromTxt(data6, path_fbank + filename)
    elif filename[:11] == 'EMOTION_DOM':
        appendJsonFromTxt(data7, path_fbank + filename)
    elif filename[:11] == 'EMOTION_HOT':
        appendJsonFromTxt(data8, path_fbank + filename)
    elif filename[:11] == 'EMOTION_ELA':
        appendJsonFromTxt(data9, path_fbank + filename)
    elif filename[:11] == 'EMOTION_INT':
        appendJsonFromTxt(data10, path_fbank + filename)
    elif filename[:11] == 'EMOTION_HAP':
        appendJsonFromTxt(data11, path_fbank + filename)
    elif filename[:11] == 'EMOTION_PAN':
        appendJsonFromTxt(data12, path_fbank + filename)
    elif filename[:11] == 'EMOTION_SAD':
        appendJsonFromTxt(data13, path_fbank + filename)
    else:
        print('this filename was not specified in if statement: ', filename)

In [96]:
print(len(data0))
print(len(data1))
print(len(data2))
print(len(data3))
print(len(data4))
print(len(data5))
print(len(data6))
print(len(data7))
print(len(data8))
print(len(data9))
print(len(data10))
print(len(data11))
print(len(data12))
print(len(data13))

103
198
270
280
127
288
134
249
83
81
115
96
88
123


In [105]:
X = data0+data1+data2+data3+data4+data5+data6+data7+data8+data9+data10+data11+data12+data13
y = [0]*len(data0)+[1]*len(data1)+[2]*len(data2)+[3]*len(data3)+[4]*len(data4)+[5]*len(data5)+\
    [6]*len(data6)+[7]*len(data7)+[8]*len(data8)+[9]*len(data9)+[10]*len(data10)+\
    [11]*len(data11)+[12]*len(data12)+[13]*len(data13)

In [107]:
print(len(X), len(y), Counter(y))

2235 2235 Counter({5: 288, 3: 280, 2: 270, 7: 249, 1: 198, 6: 134, 4: 127, 13: 123, 10: 115, 0: 103, 11: 96, 12: 88, 8: 83, 9: 81})


In [117]:
X_train, y_train, X_valid, y_valid, X_test, y_test = splitByCV(np.array(X), np.array(y), rand=0)

y_train:  Counter({5: 235, 2: 225, 3: 225, 7: 199, 1: 158, 6: 103, 4: 95, 13: 94, 10: 91, 0: 88, 11: 76, 8: 67, 12: 66, 9: 66})
y_valid:  Counter({3: 29, 5: 28, 2: 25, 7: 25, 4: 18, 6: 17, 1: 16, 13: 13, 12: 12, 11: 9, 9: 9, 8: 8, 10: 8, 0: 6})
y_test:  Counter({3: 26, 5: 25, 7: 25, 1: 24, 2: 20, 10: 16, 13: 16, 4: 14, 6: 14, 11: 11, 12: 10, 0: 9, 8: 8, 9: 6})


In [122]:
X_train.shape

(1788, 500, 24)

In [123]:
X_valid.shape

(223, 500, 24)

In [120]:
X_test.shape

(224, 500, 24)

In [124]:
X_train_frames10, y_train_frames10 = aggregateDataByFrames(X_train, y_train, 10)
X_train_frames20, y_train_frames20 = aggregateDataByFrames(X_train, y_train, 20)
X_train_frames25, y_train_frames25 = aggregateDataByFrames(X_train, y_train, 25)
X_train_frames50, y_train_frames50 = aggregateDataByFrames(X_train, y_train, 50)

X_valid_frames10, y_valid_frames10 = aggregateDataByFrames(X_valid, y_valid, 10)
X_valid_frames20, y_valid_frames20 = aggregateDataByFrames(X_valid, y_valid, 20)
X_valid_frames25, y_valid_frames25 = aggregateDataByFrames(X_valid, y_valid, 25)
X_valid_frames50, y_valid_frames50 = aggregateDataByFrames(X_valid, y_valid, 50)

X_test_frames10, y_test_frames10 = aggregateDataByFrames(X_test, y_test, 10)
X_test_frames20, y_test_frames20 = aggregateDataByFrames(X_test, y_test, 20)
X_test_frames25, y_test_frames25 = aggregateDataByFrames(X_test, y_test, 25)
X_test_frames50, y_test_frames50 = aggregateDataByFrames(X_test, y_test, 50)

ret_X.shape:  (89400, 24)    ret_y.shape:  (89400,)
ret_X.shape[0]/n_samples:  50.0
ret_X.shape:  (44700, 24)    ret_y.shape:  (44700,)
ret_X.shape[0]/n_samples:  25.0
ret_X.shape:  (35760, 24)    ret_y.shape:  (35760,)
ret_X.shape[0]/n_samples:  20.0
ret_X.shape:  (17880, 24)    ret_y.shape:  (17880,)
ret_X.shape[0]/n_samples:  10.0
ret_X.shape:  (11150, 24)    ret_y.shape:  (11150,)
ret_X.shape[0]/n_samples:  50.0
ret_X.shape:  (5575, 24)    ret_y.shape:  (5575,)
ret_X.shape[0]/n_samples:  25.0
ret_X.shape:  (4460, 24)    ret_y.shape:  (4460,)
ret_X.shape[0]/n_samples:  20.0
ret_X.shape:  (2230, 24)    ret_y.shape:  (2230,)
ret_X.shape[0]/n_samples:  10.0
ret_X.shape:  (11200, 24)    ret_y.shape:  (11200,)
ret_X.shape[0]/n_samples:  50.0
ret_X.shape:  (5600, 24)    ret_y.shape:  (5600,)
ret_X.shape[0]/n_samples:  25.0
ret_X.shape:  (4480, 24)    ret_y.shape:  (4480,)
ret_X.shape[0]/n_samples:  20.0
ret_X.shape:  (2240, 24)    ret_y.shape:  (2240,)
ret_X.shape[0]/n_samples:  10.0


In [125]:
io.savemat(path+'emotion-fbanks-frames10.mat', {'X_train': X_train_frames10, 'y_train': y_train_frames10,
       'X_valid': X_valid_frames10, 'y_valid': y_valid_frames10,
       'X_test': X_test_frames10, 'y_test': y_test_frames10} )

io.savemat(path+'emotion-fbanks-frames20.mat', {'X_train': X_train_frames20, 'y_train': y_train_frames20,
       'X_valid': X_valid_frames20, 'y_valid': y_valid_frames20,
       'X_test': X_test_frames20, 'y_test': y_test_frames20} )

io.savemat(path+'emotion-fbanks-frames25.mat', {'X_train': X_train_frames25, 'y_train': y_train_frames25,
       'X_valid': X_valid_frames25, 'y_valid': y_valid_frames25,
       'X_test': X_test_frames25, 'y_test': y_test_frames25} )

io.savemat(path+'emotion-fbanks-frames50.mat', {'X_train': X_train_frames50, 'y_train': y_train_frames50,
       'X_valid': X_valid_frames50, 'y_valid': y_valid_frames50,
       'X_test': X_test_frames50, 'y_test': y_test_frames50} )

In [126]:
data50 = io.loadmat(path+'emotion-fbanks-frames50.mat')

## Emotion-all data

In [41]:
path = 'data/audio-emotion/Splits/Emotions-all/'
path_summary = 'data/audio-emotion/Splits/Emotions-summary/'

In [46]:
y_train = []
X_train = []
with open(path_summary+'train.txt', 'r') as f_read:
    for line in f_read:
        # line_json is list object / and line is str
        line_json = json.loads(line)
        y_train.append(line_json[0]) # add 1 label
        X_train.append(line_json[1]) # add 12K feature vector

In [47]:
Counter(y_train)

Counter({0: 1014, 2: 234, 1: 372, 4: 98, 3: 70})

In [49]:
Counter(y_valid)

Counter({0: 140, 1: 51, 2: 33, 3: 10, 4: 14})

In [50]:
Counter(y_test)

Counter({0: 112, 1: 43, 2: 25, 3: 8, 4: 11})

In [6]:
y_valid = []
X_valid = []
with open(path+'validation.txt', 'r') as f_read:
    for line in f_read:
        # line_json is list object / and line is str
        line_json = json.loads(line)
        y_valid.append(line_json[0]) # add 1 label
        X_valid.append(line_json[1]) # add 12K feature vector

In [7]:
y_test = []
X_test = []
with open(path+'test.txt', 'r') as f_read:
    for line in f_read:
        # line_json is list object / and line is str
        line_json = json.loads(line)
        y_test.append(line_json[0]) # add 1 label
        X_test.append(line_json[1]) # add 12K feature vector

In [8]:
y_train = np.array(y_train)
X_train = np.array(X_train)

y_valid = np.array(y_valid)
X_valid = np.array(X_valid)

y_test = np.array(y_test)
X_test = np.array(X_test)

In [9]:
y_train.shape

(1788,)

In [11]:
X_train.shape

(1788, 12000)

### reshape so the data format change [n_samples x 12,000] => [n_samples x 500 frames x 24 log filter banks]

In [12]:
X_train = X_train.reshape(X_train.shape[0], 500, 24)
X_valid = X_valid.reshape(X_valid.shape[0], 500, 24)
X_test = X_test.reshape(X_test.shape[0], 500, 24)

In [13]:
X_train.shape

(1788, 500, 24)

### aggregate those data according to certain number of frames, e.g., 10, 20, 25, 50

In [121]:
def aggregateDataByFrames(X, y, frames_th):
	# X = [n_samples x 500 x 24], y = [n_samples, 1], frames_th = frames threshold
	frame_cnt = 0
	ret_X = []
	ret_y = []
	temp_X = []
	for i in range(X.shape[0]):
		for j in range(X.shape[1]):
			frame_cnt += 1
			temp_X.append(X[i,j,:])
			if frame_cnt >= frames_th:
				ret_X.append(np.mean(temp_X, axis=0))
				ret_y.append(y[i])
				frame_cnt = 0
				temp_X = []
	ret_X = np.array(ret_X)
	ret_y = np.array(ret_y)
	print('ret_X.shape: ', ret_X.shape, '   ret_y.shape: ', ret_y.shape)
	print('ret_X.shape[0]/n_samples: ', ret_X.shape[0]/X.shape[0])
	return ret_X, ret_y

In [25]:
X_train_frames10, y_train_frames10 = aggregateDataByFrames(X_train, y_train, 10)
X_train_frames20, y_train_frames20 = aggregateDataByFrames(X_train, y_train, 20)
X_train_frames25, y_train_frames25 = aggregateDataByFrames(X_train, y_train, 25)
X_train_frames50, y_train_frames50 = aggregateDataByFrames(X_train, y_train, 50)

X_valid_frames10, y_valid_frames10 = aggregateDataByFrames(X_valid, y_valid, 10)
X_valid_frames20, y_valid_frames20 = aggregateDataByFrames(X_valid, y_valid, 20)
X_valid_frames25, y_valid_frames25 = aggregateDataByFrames(X_valid, y_valid, 25)
X_valid_frames50, y_valid_frames50 = aggregateDataByFrames(X_valid, y_valid, 50)

X_test_frames10, y_test_frames10 = aggregateDataByFrames(X_test, y_test, 10)
X_test_frames20, y_test_frames20 = aggregateDataByFrames(X_test, y_test, 20)
X_test_frames25, y_test_frames25 = aggregateDataByFrames(X_test, y_test, 25)
X_test_frames50, y_test_frames50 = aggregateDataByFrames(X_test, y_test, 50)

ret_X.shape:  (89400, 24)    ret_y.shape:  (89400,)
ret_X.shape[0]/n_samples:  50.0
ret_X.shape:  (44700, 24)    ret_y.shape:  (44700,)
ret_X.shape[0]/n_samples:  25.0
ret_X.shape:  (35760, 24)    ret_y.shape:  (35760,)
ret_X.shape[0]/n_samples:  20.0
ret_X.shape:  (17880, 24)    ret_y.shape:  (17880,)
ret_X.shape[0]/n_samples:  10.0
ret_X.shape:  (12400, 24)    ret_y.shape:  (12400,)
ret_X.shape[0]/n_samples:  50.0
ret_X.shape:  (6200, 24)    ret_y.shape:  (6200,)
ret_X.shape[0]/n_samples:  25.0
ret_X.shape:  (4960, 24)    ret_y.shape:  (4960,)
ret_X.shape[0]/n_samples:  20.0
ret_X.shape:  (2480, 24)    ret_y.shape:  (2480,)
ret_X.shape[0]/n_samples:  10.0
ret_X.shape:  (9950, 24)    ret_y.shape:  (9950,)
ret_X.shape[0]/n_samples:  50.0
ret_X.shape:  (4975, 24)    ret_y.shape:  (4975,)
ret_X.shape[0]/n_samples:  25.0
ret_X.shape:  (3980, 24)    ret_y.shape:  (3980,)
ret_X.shape[0]/n_samples:  20.0
ret_X.shape:  (1990, 24)    ret_y.shape:  (1990,)
ret_X.shape[0]/n_samples:  10.0


### Store the resulting files

In [35]:
io.savemat(path+'emotion-all-frames10.mat', {'X_train': X_train_frames10, 'y_train': y_train_frames10,
       'X_valid': X_valid_frames10, 'y_valid': y_valid_frames10,
       'X_test': X_test_frames10, 'y_test': y_test_frames10} )

io.savemat(path+'emotion-all-frames20.mat', {'X_train': X_train_frames20, 'y_train': y_train_frames20,
       'X_valid': X_valid_frames20, 'y_valid': y_valid_frames20,
       'X_test': X_test_frames20, 'y_test': y_test_frames20} )

io.savemat(path+'emotion-all-frames25.mat', {'X_train': X_train_frames25, 'y_train': y_train_frames25,
       'X_valid': X_valid_frames25, 'y_valid': y_valid_frames25,
       'X_test': X_test_frames25, 'y_test': y_test_frames25} )

io.savemat(path+'emotion-all-frames50.mat', {'X_train': X_train_frames50, 'y_train': y_train_frames50,
       'X_valid': X_valid_frames50, 'y_valid': y_valid_frames50,
       'X_test': X_test_frames50, 'y_test': y_test_frames50} )

In [36]:
data50 = io.loadmat(path+'emotion-all-frames50.mat')

In [40]:
Counter(y_train)

Counter({1: 372, 0: 1014, 4: 98, 3: 70, 2: 234})

In [39]:
Counter(y_train_frames10)

Counter({1: 18600, 0: 50700, 4: 4900, 3: 3500, 2: 11700})

In [37]:
y_train_frames50.transpose().shape

(17880,)

In [38]:
data50['y_train'].shape

(1, 17880)

In [ ]:
# specify configurations of available data-sets.
DATASET_CONFIGS = {
    'mnist': {'features': 32, 'seq': 1, 'classes': 10},
    'mnist28': {'features': 28, 'seq': 1, 'classes': 10},
    'opportunity': {'features': 113, 'seq': 24, 'classes': 17}, # w/o null class
    'hhar-raw': {'features': 6, 'seq': 50, 'classes': 6}, # no null class
    'hhar-noaug': {'features': 120, 'seq': 20, 'classes': 6}, # no null class
    'hhar-aug': {'features': 120, 'seq': 20, 'classes': 6}, # no null class
    'opp_thomas': {'features': 77, 'seq': 30, 'classes': 18}, # w/ null class
    'pamap2': {'features': 52, 'seq': 33, 'classes': 12}, # w/o null class
    'skoda': {'features': 60, 'seq': 33, 'classes': 10}, # w/ null class
    'usc-had': {'features': 6, 'seq': 33, 'classes': 12},
    'ninapro-db2-c10': {'features': 12, 'seq': 40, 'classes': 10}, # w/o null class
    'ninapro-db3-c10': {'features': 12, 'seq': 40, 'classes': 10}, # w/o null class
    'emotion-all-frames10': {'features': 24, 'seq': 50, 'classes': 14},
    'emotion-all-frames20': {'features': 24, 'seq': 25, 'classes': 14},
    'emotion-all-frames25': {'features': 24, 'seq': 20, 'classes': 14},
    'emotion-all-frames50': {'features': 24, 'seq': 10, 'classes': 14}
}

## reference codes

In [ ]:
cnt = 0
for i in range(0,len(a)):
    if a[i] != a_sm[i]:
        print(a[i], a_sm[i])

In [ ]:
def printStats(x):
    print('Min: ', x.min())
    print('Q25: ', np.percentile(x, q=25))
    print('Mean: ', np.mean(x))
    print('Q50: ', np.percentile(x, q=50))
    print('Q75: ', np.percentile(x, q=75))
    print('Max: ', x.max())
    print('SD: ', np.std(x))